In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
with open('../input/recipes/recipe.txt' , 'r') as file:
  data = file.read()
data=data[:10000]

In [ ]:
pip install langchain-community

In [ ]:
!pip install torch accelerate peft bitsandbytes transformers trl langchain nest_asyncio html2text playwright sentence-transformers faiss-gpu
#%pip install -U datasets


In [1]:
from trl import SFTTrainer
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training,  get_peft_model
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel


2024-07-28 20:28:04.262249: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 20:28:04.262360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 20:28:04.446211: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from torch import cuda, bfloat16
import transformers
from kaggle_secrets import UserSecretsClient
from datasets import load_dataset

user_secrets = UserSecretsClient()
token = user_secrets.get_secret("HUGGINGFACE_API_KEY")


model_id = 'mistralai/Mistral-7B-Instruct-v0.2'
dataset_name = "recipe.jsonl"
new_model = "llama-2-7b-chat-recipe"

tokenizer = AutoTokenizer.from_pretrained(model_id,token=token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Exception: data did not match any variant of untagged enum PyPreTokenizerTypeWrapper at line 40 column 3

In [ ]:
import json
from tqdm import tqdm 

with open('../input/recipe-json/layer1.json' , 'r') as file:
  data = json.load(file)


url = [] 
title=[]
# Looping through and appending the values to a newly created list 
for attr in tqdm(data):
    title.append(attr['title'])
    url.append(attr['url'])
    
def get_list_of_strings(key , json_file):
    all_ingredient_lists = []
    for recipie in tqdm(json_file):
        ingredients = recipie[key]
        ingredient_list = []
        for ingredient in ingredients:
            ingredient_list.append(ingredient['text'])

        all_ingredient_lists.append(ingredient_list)
    return all_ingredient_lists


# Using the above function and parsing out the values
ingredients_list = get_list_of_strings('ingredients' , data)
instructions_list = get_list_of_strings('instructions' , data)


In [ ]:
updated_ingredients_list = [' , '.join(ingredients_list[i]) for i in tqdm(range(0 , len(data)))] 
updated_instructions_list = [' '.join(instructions_list[i]) for i in tqdm(range(0 , len(data)))] 


In [ ]:
my_file = open('recipe.jsonl','w+')
for i in range(len(url)):
    d=dict()
    d['text']='<s>[INST]Give me recipe for'+title[i]+'[/INST]'+"Ingredients : "+updated_ingredients_list[i]+"Instructions : "+updated_instructions_list[i]+"Refrence : " + url[i]+"</s>"
    my_file.write(json.dumps(d))
my_file.close()


In [ ]:

dataset = load_dataset("json", data_files=dataset_name, split='train')


In [ ]:
# LoRA attention dimension
lora_r = 32

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/kaggle/working/results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 256

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
pip install -U transformers (>4.37.2)

In [ ]:
!pip install accelarate
!pip install transformers -U
!pip install git+https://github.com/huggingface/diffusers

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map=device_map,
    token=token
)
model.config.use_cache = False
model.config.pretraining_tp = 1


In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
trainer.train()

In [ ]:
! load_ext tensorboard
! tensorboard --logdir /kaggle/working/results/runs

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "Give the recipe of mac and cheese"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:

#chunking transformed documents

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    length_function=len,
    is_separator_regex=False,
)
chunked_documents=text_splitter.create_documents([data_use])



In [ ]:
#creating database
db= FAISS.from_documents(chunked_documents,HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
retriever=db.as_retriever(
    search_type='similarity',
    search_kwargs={'k':10}
)

finetuning is requiring huge amount of processing hence i am using Rag for this project

prompting work starts

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

text_generator_pipeline= transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=200

)

prompt_template="instruction : give the recipe and list of ingredients of food item using {context} as context" +"\ngive or improvise  recipe , list of ingreients and instructions for cooking {question} if this is a food item \n"
print(prompt_template)

In [ ]:
from langchain.chains import LLMChain

minstral_llm=HuggingFacePipeline(pipeline=text_generator_pipeline)
prompt=PromptTemplate(
    input_variables=['context','question'],
    template=prompt_template,
)
llm_chain=LLMChain(llm=minstral_llm,prompt=prompt)

In [ ]:
from langchain_core.runnables import  RunnablePassthrough

query="butter chicken"
rag_chain=(
    {"context": retriever,"question" : RunnablePassthrough()}
    | llm_chain
    
)

In [ ]:
ans=rag_chain.invoke(query)

In [ ]:
s=ans['text']

In [ ]:
print(s)